In [15]:
from __future__ import division
import numpy as np
import os, sys
import matplotlib
#matplotlib.use('Agg')
%matplotlib nbagg
import matplotlib.pyplot as plt
import pandas as pd
import subprocess
from collections import defaultdict, Counter
import pickle

def getCDF(data):
    xdata = np.sort(data)
    ydata = [i/len(xdata) for i in range(len(xdata))]
    return xdata, ydata

In [10]:
df = pd.read_csv( '../processed/features_key_srcip.csv', sep="|").drop('Unnamed: 0', 1)

### Get hostdb fields from df srcips


In [16]:
srcip_list = list(df['srcip'])
pickle.dump(srcip_list, open("../data/srcip_list.pkl", 'w'))

In [21]:
pd.read_csv('../data/hostdb.csv',
            names=['srcip','entry_type_str','hostname_str','system_type_str','os_type_str','subnet_str','interface_type_str'])

,srcip,entry_type_str,hostname_str,system_type_str,os_type_str,subnet_str,interface_type_str
0,10.6.48.10,host,win04,dell-pc,ms-windows-2012,fw-servernet-2,Ethernet
1,10.6.48.100,host,dev-webqa,virtual-machine,ms-windows-2012,fw-servernet-2,Ethernet
2,10.6.48.105,host,sdpvm200w,virtual-machine,ms-windows-2012,fw-servernet-2,Ethernet
3,10.6.48.112,host,spcacs201w,virtual-machine,ms-windows-2012,fw-servernet-2,Ethernet
4,10.6.48.116,host,socopm100w,virtual-machine,ms-windows-2008,fw-servernet-2,Ethernet
5,10.6.48.122,host,ais-mars,virtual-machine,ms-windows-2012,fw-servernet-2,Ethernet
6,10.6.48.13,host,essconsole200w,virtual-machine,ms-windows-2012,fw-servernet-2,Ethernet
7,10.6.48.131,host,aissimphony303w,virtual-machine,ms-windows-2008,fw-servernet-2,Ethernet
8,10.6.48.138,host,tmetrics,virtual-machine,ms-windows-2012,fw-servernet-2,Ethernet
9,10.6.48.14,host,ess-2008r2test,virtual-machine,ms-windows-2008,fw-servernet-2,Ethernet


# Try clustering